**Завдання 1: Збір даних із вебсайту**
   - Перейдіть у підпапку `task_1_web_scraping`.
   - Створіть Python-скрипт `web_scraper.py`, що використовує бібліотеку `BeautifulSoup` для збору заголовків новин із зазначеного вебсайту.
   - Виведіть зібрані дані у CSV-файл `news_titles.csv`.
   - Додайте файл README.md, в якому коротко опишіть призначення скрипта та як його запускати.

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import time
import sys
import json
import csv
from typing import List, Dict, Optional
import requests
from bs4 import BeautifulSoup

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; UkrinformScraper/1.0; +https://example.org/bot)"
}

def fetch_page(url: str, params: Optional[dict] = None, timeout: int = 15) -> BeautifulSoup:
    resp = requests.get(url, params=params, headers=HEADERS, timeout=timeout)
    resp.raise_for_status()
    return BeautifulSoup(resp.text, "html.parser")

def parse_headlines(soup: BeautifulSoup) -> List[Dict[str, str]]:
    """
    Сторінка містить стрічку заголовків, де кожен заголовок знаходиться в <h2><a>....
    Повертаємо список словників з 'title' і 'url'.
    """
    items = []
    # Шукаємо всі h2, в яких є <a href="...">
    for h2 in soup.select("h2 a[href]"):
        title = " ".join(h2.get_text(strip=True).split())
        href = h2.get("href")
        # Пропускаємо службові та порожні
        if not title or not href:
            continue
        # Деякі посилання можуть бути відносними
        if href.startswith("//"):
            url = "https:" + href
        elif href.startswith("/"):
            # Визначаємо базовий домен з поточної сторінки (ua/.net)
            # Простий хак: якщо в HTML є <link rel="alternate" hreflang="en" ...> – але нам достатньо знати,
            # що посилання з ukrinform.ua ведуть на той же хост.
            # Тому збираємо абсолютний URL через requests.compat.urljoin:
            from urllib.parse import urljoin
            # Відновимо origin з <base>, якщо є, або задаємо вручну нижче при виклику
            # Тут зручніше буде залишити відносний і дозбирати пізніше:
            url = href
        else:
            url = href
        items.append({"title": title, "url": url})
    return items

def absolutize(items: List[Dict[str, str]], origin: str) -> List[Dict[str, str]]:
    from urllib.parse import urljoin
    for it in items:
        it["url"] = urljoin(origin, it["url"])
    return items

def get_latest_ukrinform(n_pages: int = 1, lang: str = "ua", delay_sec: float = 0.8) -> List[Dict[str, str]]:
    """
    n_pages — скільки сторінок пройти (1 = лише перша).
    lang: "ua" -> ukrinform.ua, "en" -> ukrinform.net.
    """
    if lang not in ("ua", "en"):
        raise ValueError("lang must be 'ua' or 'en'")
    base = "https://www.ukrinform.ua/block-lastnews" if lang == "ua" \
           else "https://www.ukrinform.net/block-lastnews"
    origin = "https://www.ukrinform.ua/" if lang == "ua" else "https://www.ukrinform.net/"
    all_items: List[Dict[str, str]] = []

    for page in range(n_pages):
        params = {"page": page} if page > 0 else None
        soup = fetch_page(base, params=params)
        items = parse_headlines(soup)
        items = absolutize(items, origin)
        all_items.extend(items)
        time.sleep(delay_sec)  # ввічлива пауза, щоб не навантажувати сайт

    # Прибрати дублікати за URL
    seen = set()
    unique: List[Dict[str, str]] = []
    for it in all_items:
        if it["url"] not in seen:
            unique.append(it)
            seen.add(it["url"])
    return unique

def main():
    import argparse
    ap = argparse.ArgumentParser(description="Збір заголовків останніх новин з ukrinform.ua / ukrinform.net")
    ap.add_argument("--pages", type=int, default=1, help="Кількість сторінок (за замовчуванням 1)")
    ap.add_argument("--lang", choices=["ua", "en"], default="ua", help="Мова сайту: ua або en (default: ua)")
    ap.add_argument("--out", choices=["json", "csv", "pretty"], default="pretty",
                    help="Формат виводу: json | csv | pretty (default: pretty)")
    args = ap.parse_args()

    try:
        data = get_latest_ukrinform(n_pages=args.pages, lang=args.lang)
    except requests.HTTPError as e:
        print(f"HTTP error: {e}", file=sys.stderr); sys.exit(1)
    except requests.RequestException as e:
        print(f"Network error: {e}", file=sys.stderr); sys.exit(2)

    if args.out == "json":
        print(json.dumps(data, ensure_ascii=False, indent=2))
    elif args.out == "csv":
        writer = csv.DictWriter(sys.stdout, fieldnames=["title", "url"])
        writer.writeheader()
        for row in data:
            writer.writerow(row)
    else:
        # pretty
        for i, row in enumerate(data, 1):
            print(f"{i:>2}. {row['title']}\n   {row['url']}\n")

if __name__ == "__main__":
    main()
!pip install requests beautifulsoup4


usage: ipykernel_launcher.py [-h] [--pages PAGES] [--lang {ua,en}]
                             [--out {json,csv,pretty}]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\5103_6\AppData\Roaming\jupyter\runtime\kernel-v351b3f69ed106f32145b27579c6bce789043d382d.json


SystemExit: 2

C:\Users\5103_6\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3680: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
!pip install requests beautifulsoup4


   ---------------------------------------- 0.0/64.7 kB ? eta -:--:--
   ------------------------------------- -- 61.4/64.7 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 64.7/64.7 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/105.1 kB ? eta -:--:--
   ---------------------------------------- 105.1/105.1 kB 6.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/161.2 kB ? eta -:--:--
   -------------------------------------- - 153.6/161.2 kB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 161.2/161.2 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/107.1 kB ? eta -:--:--
   ---------------------------------------- 107.1/107.1 kB 6.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/70.4 kB ? eta -:--:--
   ---------------------------------------- 70.4/70.4 kB ? eta 0:00:00
   ---------------------------------------- 0.0/129.8 kB ? eta -:--:--
   --------------------------


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\5103_6\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

URL = "https://www.ukrinform.ua/block-lastnews"
HEADERS = {"User-Agent": "Mozilla/5.0"}

def get_latest_headlines(n_pages: int = 1):
    all_items = []
    for page in range(n_pages):
        params = {"page": page} if page > 0 else None
        resp = requests.get(URL, headers=HEADERS, params=params, timeout=15)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        
        for h2 in soup.select("h2 a[href]"):
            title = h2.get_text(strip=True)
            link = urljoin(URL, h2.get("href"))
            all_items.append({"title": title, "url": link})
    return all_items

# приклад використання
headlines = get_latest_headlines(n_pages=1)

for i, item in enumerate(headlines, 1):
    print(f"{i}. {item['title']}\n   {item['url']}\n")


1. Українські парафехтувальники - треті у командному заліку чемпіонату світу
   https://www.ukrinform.ua/rubric-sports/4034438-ukrainski-parafehtuvalniki-treti-u-komandnomu-zaliku-cempionatu-svitu.html

2. У російській Пензі внаслідок вибухів вийшли з ладу два газогони - джерело
   https://www.ukrinform.ua/rubric-ato/4034442-u-rosijskij-penzi-vnaslidok-vibuhiv-vijsli-z-ladu-dva-gazogoni-dzerelo.html

3. У Бєлгородській області після дронової атаки загорілась нафтобаза - ЗМІ
   https://www.ukrinform.ua/rubric-world/4034440-u-belgorodskij-oblasti-pisla-dronovoi-ataki-zagorilas-naftobaza-zmi.html

4. Поліція Лондона почала розслідування після появи нового муралу Бенксі
   https://www.ukrinform.ua/rubric-culture/4034436-policia-londona-pocala-rozsliduvanna-pisla-poavi-novogo-muralu-benksi.html

5. Кінофестиваль Docudays UA оголосив прийом стрічок
   https://www.ukrinform.ua/rubric-culture/4034433-kinofestival-docudays-ua-ogolosiv-prijom-stricok.html

6. На Київщині працює ППО
   https://ww

In [6]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv

URL = "https://www.ukrinform.ua/block-lastnews"
HEADERS = {"User-Agent": "Mozilla/5.0"}

def get_latest_headlines(n_pages: int = 1):
    all_items = []
    for page in range(n_pages):
        params = {"page": page} if page > 0 else None
        resp = requests.get(URL, headers=HEADERS, params=params, timeout=15)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        
        for h2 in soup.select("h2 a[href]"):
            title = h2.get_text(strip=True)
            link = urljoin(URL, h2.get("href"))
            all_items.append({"title": title, "url": link})
    return all_items

# Збираємо дані (наприклад, перша сторінка)
headlines = get_latest_headlines(n_pages=1)

# Запис у CSV
with open("news_titles.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["title", "url"])
    writer.writeheader()
    writer.writerows(headlines)

print("✅ Дані збережено у файл news_titles.csv")


✅ Дані збережено у файл news_titles.csv
